# Gradient-boosting decision tree (GBDT)

El Gradient-boosting difiere de Adaboost en lo siguiente: 
- En lugar de asignar pesos a muestras específicas, GBDT se ajustará a un árbol de decisión sobre el error de residuos (de ahí el nombre de "gradiente") del árbol anterior. 
- Por tanto, cada árbol nuevo en el conjunto predice el error cometido por el modelo anterior en lugar de predecir el objetivo directamente.

In [ ]:
import pandas as pd
import numpy as np

# Cree un generador de números aleatorios
rng = np.random.RandomState(0)


def generate_data(n_samples=50):
    """Generate synthetic dataset. Returns `data_train`, `data_test`,
    `target_train`."""
    x_max, x_min = 1.4, -1.4
    len_x = x_max - x_min
    x = rng.rand(n_samples) * len_x - len_x / 2
    noise = rng.randn(n_samples) * 0.3
    y = x ** 3 - 0.5 * x ** 2 + noise

    data_train = pd.DataFrame(x, columns=["Feature"])
    data_test = pd.DataFrame(np.linspace(x_max, x_min, num=300),
                             columns=["Feature"])
    target_train = pd.Series(y, name="Target")

    return data_train, data_test, target_train


data_train, data_test, target_train = generate_data()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.scatterplot(x=data_train["Feature"], y=target_train, color="black",
                alpha=0.5)
_ = plt.title("Synthetic regression dataset")

In [ ]:
# Creamos un regresor de árbol de decisión. 
# Establecemos la profundidad del árbol para que el modelo resultante no subajuste a los datos.

from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=3, random_state=0)
tree.fit(data_train, target_train)

target_train_predicted = tree.predict(data_train)
target_test_predicted = tree.predict(data_test)


In [ ]:
# grafica los datos
sns.scatterplot(x=data_train["Feature"], y=target_train, color="black",
                alpha=0.5)
# grafica las predicciones
line_predictions = plt.plot(data_test["Feature"], target_test_predicted, "--")

# grafica los residuos
for value, true, predicted in zip(data_train["Feature"],
                                  target_train,
                                  target_train_predicted):
    lines_residuals = plt.plot([value, value], [true, predicted], color="red")

plt.legend([line_predictions[0], lines_residuals[0]],
           ["Fitted tree", "Residuals"])
_ = plt.title("Función de predicción junto \na errores en el conjunto de entrenamiento")

> Dado que el árbol subajusta los datos, su precisión está lejos de ser perfecta en los datos de entrenamiento.
- Podemos observar esto en la figura observando la diferencia entre predicciones y los datos reales.
- Representamos estos errores, llamados *"residuos"*, con líneas rojas.

> En un algoritmo de refuerzo de gradiente, la idea es crear un segundo árbol que, dados los mismos datos, intentará predecir los residuos en lugar del target del vector.
- Por tanto, tendremos un árbol que puede predecir los errores cometidos por el árbol inicial.

In [ ]:
# Entrenamos dicho árbol.

residuals = target_train - target_train_predicted

tree_residuals = DecisionTreeRegressor(max_depth=5, random_state=0)
tree_residuals.fit(data_train, residuals)

target_train_predicted_residuals = tree_residuals.predict(data_train)
target_test_predicted_residuals = tree_residuals.predict(data_test)

In [ ]:
sns.scatterplot(x=data_train["Feature"], y=residuals, color="black", alpha=0.5)
line_predictions = plt.plot(
    data_test["Feature"], target_test_predicted_residuals, "--")

# los residuos de los residuos predichos
for value, true, predicted in zip(data_train["Feature"],
                                  residuals,
                                  target_train_predicted_residuals):
    lines_residuals = plt.plot([value, value], [true, predicted], color="red")

plt.legend([line_predictions[0], lines_residuals[0]],
           ["Fitted tree", "Residuals"], bbox_to_anchor=(1.05, 0.8),
           loc="upper left")
_ = plt.title("Predicción de los residuos anteriores")

> Vemos que este nuevo árbol solo logra que se ajusten alguno de los residuos.
- Nos centraremos en una muestra específica del conjunto de entrenamiento (es decir, sabemos que la muestra estará bien predicha usando dos árboles sucesivos).
- Usaremos esta muestra para explicar cómo se combinan las predicciones de ambos árboles.

Primero seleccionemos esta muestra en data_train.

In [ ]:
sample = data_train.iloc[[-2]]
x_sample = sample['Feature'].iloc[0]
target_true = target_train.iloc[-2]
target_true_residual = residuals.iloc[-2]

**Graficamos la información anterior y resaltemos nuestra muestra de interés.**

In [ ]:
# graficar los datos originales y la predicción del primer árbol de decisión.

# graficar la información anterior:
#   * el conjunto de datos
#   * las predicciones
#   * los residuos

sns.scatterplot(x=data_train["Feature"], y=target_train, color="black",
                alpha=0.5)
plt.plot(data_test["Feature"], target_test_predicted, "--")
for value, true, predicted in zip(data_train["Feature"],
                                  target_train,
                                  target_train_predicted):
    lines_residuals = plt.plot([value, value], [true, predicted], color="red")

# Resaltar la muestra de interés
plt.scatter(sample, target_true, label="Muestra de interés",
            color="tab:orange", s=200)
plt.xlim([-1, 0])
plt.legend(bbox_to_anchor=(1.05, 0.8), loc="upper left")
_ = plt.title("Tree predictions")

In [ ]:
# Ahora, graficamos la información de los residuos. 
# Trazaremos los residuos calculados a partir del primer árbol de decisión y mostraremos las predicciones residuales.

# graficar la información anterior:
# * Los residuos generados por el primer árbol
# * Las predicciones residuales
# * Los residuos de las predicciones residuales

sns.scatterplot(x=data_train["Feature"], y=residuals,
                color="black", alpha=0.5)
plt.plot(data_test["Feature"], target_test_predicted_residuals, "--")
for value, true, predicted in zip(data_train["Feature"],
                                  residuals,
                                  target_train_predicted_residuals):
    lines_residuals = plt.plot([value, value], [true, predicted], color="red")

# Resaltar la muestra de interés
plt.scatter(sample, target_true_residual, label="Muestra de interés",
            color="tab:orange", s=200)
plt.xlim([-1, 0])
plt.legend()
_ = plt.title("Predicción de los residuos")

- Para nuestra muestra de interés, nuestro árbol inicial está cometiendo un error (pequeño residuo).
- Al ajustar el segundo árbol, el residuo en este caso está perfectamente ajustado y predicho.
- Verificamos cuantitativamente esta predicción usando el árbol ajustado.

In [ ]:
# Primero, verifiquemos la predicción del árbol inicial y comparemos con el valor real.

print(f"Valor real a predecir para "
      f"f(x={x_sample:.3f}) = {target_true:.3f}")

y_pred_first_tree = tree.predict(sample)[0]
print(f"Predicción del primer árbol de decisión para x={x_sample:.3f}: "
      f"y={y_pred_first_tree:.3f}")
print(f"Error del árbol: {target_true - y_pred_first_tree:.3f}")

In [ ]:
# Como observamos visualmente, tenemos un pequeño error. 
# Ahora, podemos usar el segundo árbol para tratar de predecir este residual.

print(f"Predicción de la residuo para x={x_sample:.3f}: "
      f"{tree_residuals.predict(sample)[0]:.3f}")

- Vemos que nuestro segundo árbol es capaz de predecir el residuo exacto (error) de nuestro primer árbol.
- Por lo tanto, podemos predecir el valor de x sumando la predicción de todos los árboles en el conjunto.

In [ ]:
y_pred_first_and_second_tree = (
    y_pred_first_tree + tree_residuals.predict(sample)[0]
)
print(f"Predicción del primer y segundo árbol de decisión combinados para "
      f"x={x_sample:.3f}: y={y_pred_first_and_second_tree:.3f}")
print(f"Error del árbol: {target_true - y_pred_first_and_second_tree:.3f}")

- Elegimos una muestra para la cual solo dos árboles eran suficientes para hacer la predicción perfecta.
- Sin embargo, vimos en la gráfica anterior que dos árboles no eran suficientes para corregir los residuos de todas las muestras.
- Por lo tanto, se debe agregar varios árboles al conjunto para corregir con éxito el error (es decir, el segundo árbol corrige el error del primer árbol; el tercer árbol corrige el error del segundo árbol, etc.).

In [ ]:
# Compararemos el rendimiento de generalización del random-forest y el gradient boosting en el conjunto de datos de viviendas de California.

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import cross_validate

data, target = fetch_california_housing(return_X_y=True, as_frame=True)
target *= 100 

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gradient_boosting = GradientBoostingRegressor(n_estimators=200)
cv_results_gbdt = cross_validate(
    gradient_boosting, data, target, scoring="neg_mean_absolute_error",
    n_jobs=2,
)

In [ ]:
print("Gradient Boosting Decision Tree")
print(f"Error absoluto medio por validación cruzada: "
      f"{-cv_results_gbdt['test_score'].mean():.3f} ± "
      f"{cv_results_gbdt['test_score'].std():.3f} k$")
print(f"Tiempo de ajuste promedio: "
      f"{cv_results_gbdt['fit_time'].mean():.3f} seconds")
print(f"Tiempo de puntaje promedio: "
      f"{cv_results_gbdt['score_time'].mean():.3f} seconds")

In [ ]:
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor(n_estimators=200, n_jobs=2)
cv_results_rf = cross_validate(
    random_forest, data, target, scoring="neg_mean_absolute_error",
    n_jobs=2,
)

In [ ]:
print("Random Forest")
print(f"Error absoluto medio por validación cruzada: "
      f"{-cv_results_rf['test_score'].mean():.3f} ± "
      f"{cv_results_rf['test_score'].std():.3f} k$")
print(f"Tiempo de ajuste promedio: "
      f"{cv_results_rf['fit_time'].mean():.3f} seconds")
print(f"Tiempo de puntaje promedio: "
      f"{cv_results_rf['score_time'].mean():.3f} seconds")

> - En términos de **rendimiento de cálculo, el bosque puede ser paralelizado** y se beneficiará al usar múltiples núcleos de la CPU.
> - En términos de **rendimiento de puntuación, ambos algoritmos conducen a resultados muy cercanos**.
> - Sin embargo, vemos que el **gradient boosting es muy rápido** en comparación con el bosque aleatorio. Debido al hecho de que el gradient boosting usa árboles poco profundos.